# Advanced Scheduling

- We Could Add Priorities to favor some processes over others
- Assign each process a priority
- Run higher priority processes first, round-robin processes of equal priority Can be preemptive or non-preemptive

### Priorities Can Be Assigned an Integer

We can pick a lower, or higher number, to mean high priority

In Linux -20 is the highest priority, 19 is the lowest

We may lead processes to "starvation" of lower priority processes if there’s a lot of higher priority processes

One solution is to have the OS dynamically change the priority.  
Older processes that haven’t been executed in a long time increase priority

### --> Priority Inversion is a New Issue

We can **accidentally change** the priority of a low priority process to a high one  
This is caused by dependencies, e.g. a high priority depends a low priority

One solution is **priority inheritance**  
- Inherit the highest priority of the waiting processes  
- Chain together multiple inheritances if needed  
- Revert back to the original priority after dependency

If you have a clear separation of **Foreground processes** and **Background processes** 

Unix background process when: process group ID different from its terminal group ID --> You're a background task (cannot receive input - terminal - from user)

- Foreground processes are interactable and need good response time --> more priority
- Background processes may not need good response time, just throughput --> less priority

### We can use Multiple Queues

We could create different queues for foreground and background processes: (Example:)
- Foreground uses RR
- Background uses FCFS

--> We have to schedule between queues
- RR between the queues
- Use a priority for each queue

..... The scheduling can get more and more complicated (There’s no “right answer”, only trade-offs)

We’ll assume <mark>**symmetric multiprocessing (SMP)**</mark>
- All CPUs are connected to the same physical memory
- The CPUs have their own private cache (at least the lowest levels)

### 1.  Use the Same Scheduling for All CPUs

There’s still only one scheduler: It just keeps adding processes while there’s available CPUs

--> Keep scheduling processes across CPUs

Advantage: 
- Good CPU utilization  
- Fair to all processes

Disadvantages:
- Not scalable (**everything blocks on global scheduler**) (2 CPUs cannot ask questions to the memory at the same tiem)
- Poor cache locality (If a process gets RR and may move to a different CPU, different cache access and there will be no locality)

This was the approach in Linux 2.4

### 2. Create Per-CPU Schedulers

When there’s a new process, assign it to a CPU  
One strategy is to assign it to the CPU with the lowest number of processes  
**--> Process will not change CPU**

Advantages:
- Easy to implement
- Scalable (there’s no blocking on a resource - same scheduler in 1 CPU)
- Good cache locality (still on the same CPU)

Disadvantages:
- Load imbalance: Some CPUs may have less processes, or less intensive ones

### 2.1. Compromise between Global and Per-CPU

Keep a global scheduler that can rebalance per-CPU queues  
If a CPU is idle, take a process from another CPU (**work stealing** - "process stealing")

Disadvantage: If a process changes core, the cache locality will be reset

--> If we really want a process to have better preformance:

Use **processor affinity**  
The preference of a process to be scheduled on the same core

This is a simplified version of the O(1) scheduler in Linux 2.6

### 3. “Gang” Scheduling

Multiple processes may need to be scheduled simultaneously

The scheduler on each CPU cannot be completely independent

“Gang Scheduling” (Coscheduling)  
Allows you to run a set of processes simultaneously (acting as a unit)  

This requires a global context-switch across all CPUs

### 4. Real-Time Scheduling

Real-time means there are time constraints (e.g. autopilot)

- A hard real-time system: Required to guarantee a task completes within a certain amount of time
- A soft real-time system: Critical processes have a higher priority and the deadline is met in practice

Linux is an example of soft real-time

### 5. Linux Implements FCFS and RR Scheduling for real-time processes (Same schedule for all CPUs)
### It uses (Per-CPU Schedulers) for Normal processes

You can search the source tree: FCFS (SCHED_FIFO) and RR (SCHED_RR)

Use a multilevel queue scheduler for processes with the same priority  
Also let the OS dynamically adjust the priority

Soft real-time processes: Always schedule the highest priority processes first

Normal processes: Adjust the priority based on aging

#### **Soft Real-Time Processes Are Always Prioritized**

The soft real-time scheduling policy will either be SCHED_FIFO or SCHED_RR (because they are predictable)  
There are 100 static priority levels (0—99) (levels, not actual Linux priority)

Normal scheduling policies apply to the other processes (SCHED_NORMAL)  
By default the priority is 0  
Priority ranges from [−20, 19]  

Processes can change their own priorities with system calls:  
`nice`, `sched_setscheduler`|

**Linux tries to unifies all the normal and real-time process**

--> This is the number that's shown through Linux

![Alt text](images/image13.png)

If you read the PRI of all processes in Linux, showing "RT" means -100

### Linux Scheduler Evolution

2.4—2.6, a O(N) global queue  
Simple, but poor performance with multiprocessors and many processes

(Previous: 5.)  
2.6—2.6.22, a per-CPU run queue, O(1) scheduler  
Complex to get right, interactivity had issues  
No guarantee of fairness

2.6.23—Present, the **completely fair scheduler (CFS)**  
Fair, and allows for good interactivity

### The O(1) Scheduler Has Issues with Modern Processes

Foreground and background processes are a good division  
Easier with a terminal, less so with GUI processes

Now the kernel has to detect interactive processes with heuristics  
Processes that sleep a lot may be more interactive  
This is ad hoc, and could be unfair

How would we introduce fairness for different priority processes?  
Use different size time slices  
The higher the priority, the larger the time slice  
There are also situations where this ad hoc solution could be unfair

### Ideal Fair Scheduling

Assume you have an infinitely small time slice  
If you have n processes, each runs at 1/n rate